In [1]:
import os
import itertools
import time
import json
import tqdm

import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.model_selection import KFold, train_test_split

# Feature scaling, required for non-tree-based models
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats.mstats import winsorize

# Encoding categorical data for non-tree-based models
from sklearn.preprocessing import OneHotEncoder


In [2]:
mean_submit_52 = pd.read_csv('../datasets/mean_submission_52.csv', low_memory = True, thousands = ',')

In [4]:
mean_submit_52

,sku_id,week1,week2,week3,week4,week5
0,SKDtK67I,1.076923,1.076923,1.076923,1.076923,1.076923
1,SKpLKkIS,0.673077,0.673077,0.673077,0.673077,0.673077
2,SK2vQMpX,9.423077,9.423077,9.423077,9.423077,9.423077
3,SKY0RuBE,1.884615,1.884615,1.884615,1.884615,1.884615
4,SKvr2o3y,0.403846,0.403846,0.403846,0.403846,0.403846
5,SK4VKk3b,0.807692,0.807692,0.807692,0.807692,0.807692
6,SKDbZjmX,1.211538,1.211538,1.211538,1.211538,1.211538
7,SK9wkGUJ,0.403846,0.403846,0.403846,0.403846,0.403846
8,SKjQriLy,0.403846,0.403846,0.403846,0.403846,0.403846
9,SKGy1eQD,1.884615,1.884615,1.884615,1.884615,1.884615


In [19]:
# 预测总销量
# 259845.879631	242683.127875	291968.605433	364250.101998	396110.16244

# 最近52天平均销量
# 215658.96153846156

# w1 = 259845.879631
# w2 = 242683.127875
# w3 = 291968.605433
# w4 = 364250.101998
# w5 = 396110.16244
# total = 215658.96153846156

#zonghe = +78
# w1 = 1.12
# w2 = 1.08
# w3 = 1.16
# w4 = 1.20
# w5 = 1.22
# total = 1
# cons = 0.5

# w1 = 1.17
# w2 = 1.13
# w3 = 1.21
# w4 = 1.25
# w5 = 1.27
# total = 1

# w1 = 1.06
# w2 = 1.04
# w3 = 1.08
# w4 = 1.10
# w5 = 1.11
# total = 1
# cons = 0.5



#刘亮
# w1 = 0.95
# w2 = 0.87
# w3 = 1.26
# w4 = 1.46
# w5 = 0.70
# total = 1
# cons = 0

#默认
# w1 = 0.90
# w2 = 0.86
# w3 = 1.16
# w4 = 1.20
# w5 = 0.73
# total = 1
# cons = 0

#唯朗1
#zonghe = +36
# w1 = 1
# w2 = 0.94
# w3 = 1.22
# w4 = 1.32
# w5 = 0.88
# total = 1
# cons = 0



# 唯朗2群1
# w1 = 1.05
# w2 = 0.85
# w3 = 1.20
# w4 = 1.45
# w5 = 0.80

# w1 = 1.12
# w2 = 1.08
# w3 = 1.16
# w4 = 1.20
# w5 = 1.22

#唯朗2
#zonghe = +35
# w1 = 1.09
# w2 = 0.85
# w3 = 1.21
# w4 = 1.42
# w5 = 0.78
# total = 1
# cons = 0

#0.10727607000

# # #LL2
# # #zong = +34
# w1 = 1.08
# w2 = 0.82
# w3 = 1.28
# w4 = 1.44
# w5 = 0.72
# total = 1
# cons = 0

#0.10740604


#默认2
#zong = +35
# w1 = 1.04
# w2 = 0.76
# w3 = 1.34
# w4 = 1.56
# w5 = 0.65
# total = 1
# cons = 0

#0.10743611

# #SXX
# w1 = 0.80 #验证 w1 = 0.90猜想
# w2 = 1.05
# w3 = 1.07
# w4 = 1.13
# w5 = 1.08
# total = 1
# cons = 0

#Final 唯朗
# w1 = 1.085
# w2 = 0.79
# w3 = 1.31
# w4 = 1.50
# w5 = 0.70
# total = 1
# cons = 0

# #Final 默认
# w1 = 0.90
# w2 = 1.05
# w3 = 1.29
# w4 = 1.53
# w5 = 0.75
# total = 1
# cons = 0

# #Final LL
w1 = 0.92521
w2 = 1.08626
w3 = 1.18523
w4 = 1.551027
w5 = 1.01123
total = 1
cons = 0

In [20]:
mean_submit_52 = pd.read_csv('../datasets/mean_submission_52.csv', low_memory = True, thousands = ',')

In [21]:
result = mean_submit_52
# result['week1'] = result['week1'] + cons
# result['week2'] = result['week2'] + cons
# result['week3'] = result['week3'] + cons
# result['week4'] = result['week4'] + cons
# result['week5'] = result['week5'] + cons
result['week1'] = result['week1'] * w1 / total
result['week2'] = result['week2'] * w2 / total
result['week3'] = result['week3'] * w3 / total
result['week4'] = result['week4'] * w4 / total
result['week5'] = result['week5'] * w5 / total

In [22]:
result

,sku_id,week1,week2,week3,week4,week5
0,SKDtK67I,0.996380,1.169818,1.276402,1.670337,1.089017
1,SKpLKkIS,0.622737,0.731137,0.797751,1.043960,0.680636
2,SK2vQMpX,8.718325,10.235912,11.168513,14.615447,9.528898
3,SKY0RuBE,1.743665,2.047182,2.233703,2.923089,1.905780
4,SKvr2o3y,0.373643,0.438682,0.478651,0.626376,0.408381
5,SK4VKk3b,0.747285,0.877364,0.957301,1.252753,0.816763
6,SKDbZjmX,1.120928,1.316046,1.435952,1.879129,1.225144
7,SK9wkGUJ,0.373643,0.438682,0.478651,0.626376,0.408381
8,SKjQriLy,0.373643,0.438682,0.478651,0.626376,0.408381
9,SKGy1eQD,1.743665,2.047182,2.233703,2.923089,1.905780


In [23]:
result['week1'] = result['week1'].apply(np.round)
result['week2'] = result['week2'].apply(np.round)
result['week3'] = result['week3'].apply(np.round)
result['week4'] = result['week4'].apply(np.round)
result['week5'] = result['week5'].apply(np.round)

In [24]:
result

,sku_id,week1,week2,week3,week4,week5
0,SKDtK67I,1.0,1.0,1.0,2.0,1.0
1,SKpLKkIS,1.0,1.0,1.0,1.0,1.0
2,SK2vQMpX,9.0,10.0,11.0,15.0,10.0
3,SKY0RuBE,2.0,2.0,2.0,3.0,2.0
4,SKvr2o3y,0.0,0.0,0.0,1.0,0.0
5,SK4VKk3b,1.0,1.0,1.0,1.0,1.0
6,SKDbZjmX,1.0,1.0,1.0,2.0,1.0
7,SK9wkGUJ,0.0,0.0,0.0,1.0,0.0
8,SKjQriLy,0.0,0.0,0.0,1.0,0.0
9,SKGy1eQD,2.0,2.0,2.0,3.0,2.0


In [25]:
result.to_csv('../submissions/mean_submission_LL33333.csv')

In [24]:
w1 / total
# w2 / total
# w3 / total
# w4 / total
# w5 / total

1.2048925663803585

In [25]:
w2 / total

1.1253097304362139

In [26]:
w3 / total

1.3538440663451357

In [27]:
w4 / total

1.6890098116003311

In [28]:
w5 / total

1.8367433452068995